# Mean Average Recall Report

This notebooks provides an overview for using and understanding the mean average recall report check.

**Structure:**

- [What is the purpose of the check?](#purpose)
- [Generate data & model](#generate)
- [Defining the Prediction Formatter](#pred_format)
- [Run the check](#run_check)
- [Define a condition](#define_condition)

## What is the purpose of the check? <a name='purpose'></a>

The mean average recall report evaluates the mean average recall metric on the given model and data. 
The check only works on object detection as it computes the value with the detections per image amount and Area size.

### Imports

In [1]:
import numpy as np

from deepchecks.vision.base import VisionData
from deepchecks.vision.checks.performance import MeanAverageRecallReport
from deepchecks.vision.utils.detection_formatters import DetectionLabelFormatter, DetectionPredictionFormatter
from deepchecks.vision.datasets.detection import coco

### Generate Data and Model <a name='generate'></a>

We generate a sample dataset of 128 images from the [COCO dataset](https://cocodataset.org/#home), and using the [YOLOv5 model](https://github.com/ultralytics/yolov5).

For the label formatter - our dataset returns exactly the accepted format, so our formatting function is the simple `lambda x: x` function.

In [2]:
yolo = coco.load_model(pretrained=True)

test_ds = coco.load_dataset(train=False, object_type='VisionData')

### Defining the prediction formatter <a name='pred_format'></a>
The COCO module implemented in deepchecks already contains a transformation function to convert the yolo detection to the correct format of [x, y, w, h, confidence, class_id].

In [3]:
from deepchecks.vision.utils.detection_formatters import DetectionPredictionFormatter

det_formatter = DetectionPredictionFormatter(coco.yolo_prediction_formatter)

### Run the check <a name='run_check'></a>

In [4]:
check = MeanAverageRecallReport()
result = check.run(test_ds, yolo, prediction_formatter=det_formatter)
result

/Users/itaygabbay/Deepchecks/deepchecks/venv/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning:

User provided device_type of 'cuda', but CUDA is not available. Disabling



Mean Average Recall Report Summarize mean average recall metrics on a dataset and model per detections and area range. Read More... Additional Outputs 
 
 
 
   
 AR@1 (%) 
 AR@10 (%) 
 AR@100 (%) 
 
 
 Area size 
   
   
   
 
 
 
 
 All 
 0.30 
 0.38 
 0.39 
 
 
 Small (area < 32^2) 
 0.09 
 0.17 
 0.19 
 
 
 Medium (32^2 < area < 96^2) 
 0.32 
 0.39 
 0.40 
 
 
 Large (area < 96^2) 
 0.43 
 0.48 
 0.49

### Observe the check’s output

The result value is a dataframe that has the average recall score per each area range and IoU.

In [5]:
result.value

AR@1 (%)  AR@10 (%)  AR@100 (%)
Area size                                                   
All                          0.299843   0.379349    0.387681
Small (area < 32^2)          0.091908   0.167694    0.194361
Medium (32^2 < area < 96^2)  0.315645   0.392778    0.403441
Large (area < 96^2)          0.426184   0.483006    0.488989

## Define a condition <a name='define_condition'></a>

We can define a condition that enforce our model's average recall score is not less than a given threshold

In [6]:
check = MeanAverageRecallReport().add_condition_test_average_recall_not_less_than(0.4)
result = check.run(test_ds, yolo, prediction_formatter=det_formatter)
result.show(show_additional_outputs=False)

/Users/itaygabbay/Deepchecks/deepchecks/venv/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning:

User provided device_type of 'cuda', but CUDA is not available. Disabling



Mean Average Recall Report Summarize mean average recall metrics on a dataset and model per detections and area range. Read More... Conditions Summary 
 
 
 
 Status 
 Condition 
 More Info 
 
 
 
 
 ✖ 
 Scores are not less than 0.4 
 Found scores below threshold:
{'All': {'AR@1 (%)': '0.3'}, 'Small (area < 32^2)': {'AR@10 (%)': '0.168'}}